In [13]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel

df = pd.read_csv('data/cards_clean.csv')

In [38]:
from scripts_and_functions.functions import preprocess_text

df['text'] = preprocess_text(df['oracle_text'])
df = df.dropna(subset=['text'])

In [17]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model = BertModel.from_pretrained('bert-base-uncased')

model.eval()

100%|██████████| 440473133/440473133 [00:14<00:00, 31443754.29B/s]


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [53]:
samples = df[['name','text']].iloc[:1000]

In [55]:
from scripts_and_functions.functions import bert_embedding

samples['embeddings'] = samples['text'].apply(bert_embedding)

In [58]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn.fit(samples['embeddings'].tolist())

NearestNeighbors(algorithm='kd_tree')

In [84]:
test = samples.iloc[193]
test[['name','text']]

name                                             Gemrazer
text    mutate {1}{g}{g} (if you cast this spell for i...
Name: 205, dtype: object

In [95]:
distances, indices = nn.kneighbors([test['embeddings']])

In [96]:
for rank, index in enumerate(indices[0], start=1):
    print(f"Rank: {rank}, Index: {index}, Distance: {distances[0][rank-1]}")

Rank: 1, Index: 193, Distance: 0.0
Rank: 2, Index: 908, Distance: 4.381440674830688
Rank: 3, Index: 86, Distance: 4.8758160991718364
Rank: 4, Index: 395, Distance: 5.614547255984955
Rank: 5, Index: 460, Distance: 5.69507874649073


In [97]:
test['text']

'mutate {1}{g}{g} (if you cast this spell for its mutate cost, put it over or under target non-human creature you own. they mutate into the creature on top plus all abilities from under it.) reach, trample whenever this creature mutates, destroy target artifact or enchantment an opponent controls.'

In [98]:
samples.iloc[indices[0][1]]['text']

'mutate {1}{w/u}{r}{r} (if you cast this spell for its mutate cost, put it over or under target non-human creature you own. they mutate into the creature on top plus all abilities from under it.) flying, first strike whenever this creature mutates, you may cast target noncreature card with mana value 3 or less from your graveyard without paying its mana cost.'